In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import shap
import matplotlib.pyplot as plt

file_path = "adult.csv"

# Caricamento del dataset e gestione dei valori mancanti
try:
    df = pd.read_csv(file_path, na_values='?')
except FileNotFoundError:
    print(f"Errore: File '{file_path}' non trovato. Assicurati che il file sia presente.")
    exit()

# Eliminazione delle righe con valori mancanti
df = df.dropna()

# Preprocessing iniziale delle feature
# Mappatura delle colonne 'gender' e 'income' a valori numerici
df['gender'] = df['gender'].map({'Male': 0, 'Female': 1})
df['high_income'] = df['income'].map({'<=50K': 0, '>50K': 1})
df.drop('income', axis=1, inplace=True)

# Raggruppamento della colonna 'native-country'
df['native-country'] = df['native-country'].apply(lambda x: 'Other' if x != 'United-States' else x)

# Eliminazione di colonne non necessarie o ridondanti
df.drop('education', axis=1, inplace=True)
df.drop('fnlwgt', axis=1, inplace=True)

# Separazione delle feature (X) e del target (y)
X = df.drop('high_income', axis=1)
y = df['high_income']

# Divisione del dataset in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identificazione delle feature categoriche e numeriche
categorical_features_to_encode = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
numerical_features = ['age', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'gender']

# Creazione di copie per l'encoding e la scalatura
X_train_processed = X_train.copy()
X_test_processed = X_test.copy()

# Applicazione dell'Ordinal Encoding alle feature categoriche
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_train_processed[categorical_features_to_encode] = ordinal_encoder.fit_transform(X_train[categorical_features_to_encode])
X_test_processed[categorical_features_to_encode] = ordinal_encoder.transform(X_test[categorical_features_to_encode])

# Scalatura delle feature numeriche usando StandardScaler
scaler = StandardScaler()
X_train_processed[numerical_features] = scaler.fit_transform(X_train_processed[numerical_features])
X_test_processed[numerical_features] = scaler.transform(X_test_processed[numerical_features])

# Inizializzazione e addestramento del modello LightGBM
print("\nInizio addestramento del modello LightGBM...")
lgbm_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, verbose=-1)
lgbm_model.fit(X_train_processed, y_train)
print("Addestramento completato.")

# Effettuare predizioni sul test set
print("\nEffettuo predizioni sul test set...")
y_pred_lgbm = lgbm_model.predict(X_test_processed)
y_pred_proba_lgbm = lgbm_model.predict_proba(X_test_processed)[:, 1]

# Valutazione del Modello
print("\n--- Valutazione del Modello LightGBM ---")
accuracy = accuracy_score(y_test, y_pred_lgbm)
print(f"Accuratezza: {accuracy:.4f}")
roc_auc = roc_auc_score(y_test, y_pred_proba_lgbm)
print(f"ROC AUC Score: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lgbm, target_names=['<=50K', '>50K']))
print("\nMatrice di Confusione:")
cm = confusion_matrix(y_test, y_pred_lgbm)
print(pd.DataFrame(cm, index=['Reale: <=50K', 'Reale: >50K'], columns=['Predetto: <=50K', 'Predetto: >50K']))

# Spiegazione del Modello con SHAP
print("\n--- Spiegazione del Modello LightGBM con SHAP ---")
explainer = shap.TreeExplainer(lgbm_model)
print("SHAP Explainer creato.")
print("Calcolo dei valori SHAP per il test set (basato su X_test_processed)...")
shap_values = explainer.shap_values(X_test_processed)
print("Calcolo dei valori SHAP completato.")

# Determinazione di expected_value e shap_values per la classe positiva
base_expected_value = explainer.expected_value
shap_values_positive_class = shap_values

if isinstance(shap_values, list) and len(shap_values) == 2:
    shap_values_positive_class = shap_values[1]
    if isinstance(base_expected_value, list) and len(base_expected_value) == 2:
        expected_value_positive_class = base_expected_value[1]
    else:
        expected_value_positive_class = base_expected_value
else:
    expected_value_positive_class = base_expected_value

# Visualizzazione dell'importanza globale delle feature (Bar Plot)
print("\nGenerazione del Summary Plot SHAP (bar)...")
plt.figure()
shap.summary_plot(shap_values_positive_class, X_test_processed, plot_type="bar", show=False)
plt.title("Importanza Globale delle Feature (LightGBM - SHAP Summary Plot - Bar)")
plt.tight_layout()
plt.show()
print("Summary Plot (bar) generato e mostrato.")

# Visualizzazione dell'importanza e dell'impatto delle feature (Dot Plot)
print("\nGenerazione del Summary Plot SHAP (dot)...")
plt.figure()
shap.summary_plot(shap_values_positive_class, X_test, feature_names=X_test_processed.columns, show=False)
plt.title("Importanza e Impatto delle Feature (LightGBM - SHAP Summary Plot - Dot)")
plt.tight_layout()
plt.show()
print("Summary Plot (dot) generato e mostrato.")

# Visualizzazione dell'impatto di una singola feature (Dependence Plot)
feature_to_plot = 'age'
if feature_to_plot in X_test_processed.columns:
    print(f"\nGenerazione del Dependence Plot SHAP per '{feature_to_plot}'...")
    plt.figure()
    shap.dependence_plot(feature_to_plot, shap_values_positive_class, X_test, feature_names=X_test_processed.columns, interaction_index="auto", show=False)
    plt.title(f"SHAP Dependence Plot per '{feature_to_plot}' (LightGBM - valori X originali/non scalati)")
    plt.tight_layout()
    plt.show()
    print(f"Dependence Plot per '{feature_to_plot}' generato e mostrato.")
else:
    print(f"\nLa feature '{feature_to_plot}' non è presente nelle colonne. Impossibile generare il dependence plot.")

# Spiegazione di predizioni individuali con Force Plot
print("\n--- Spiegazione di Predizioni Individuali con SHAP Force Plot (LightGBM) ---")
num_predictions_to_explain = 3
print(f"Generazione dei Force Plot SHAP per le prime {num_predictions_to_explain} istanze del test set...")

for i in range(min(num_predictions_to_explain, X_test_processed.shape[0])):
    print(f"\nSpiegazione per l'istanza {i} del test set:")
    try:
        instance_features_for_display = X_test.iloc[i].copy()
        if 'gender' in instance_features_for_display.index:
            instance_features_for_display['gender'] = 'Female' if instance_features_for_display['gender'] == 1 else 'Male'

        current_shap_values_instance = shap_values_positive_class[i,:]

        plt.figure(figsize=(30, 10))
        shap.force_plot(expected_value_positive_class,
                         current_shap_values_instance,
                         instance_features_for_display,
                         feature_names=X_test_processed.columns,
                         matplotlib=True, show=False, text_rotation=10)
        plt.title(f"SHAP Force Plot per Istanza {i} (LightGBM - Valori Feature Originali/Mappati)")
        plt.tight_layout()
        plt.show()
        print(f"Force Plot (Matplotlib) per l'istanza {i} generato e mostrato.")

    except Exception as e:
        print(f"Errore durante la generazione del force plot Matplotlib per l'istanza {i}: {e}")

if X_test_processed.shape[0] < num_predictions_to_explain:
    print(f"\nAttenzione: richieste {num_predictions_to_explain} spiegazioni (force plot), ma il test set ha solo {X_test_processed.shape[0]} istanze.")

# Spiegazione di più predizioni con Decision Plot
print("\n--- Spiegazione di Predizioni Multiple con SHAP Decision Plot (LightGBM) ---")
num_decision_plots = 5
if X_test_processed.shape[0] >= num_decision_plots:
    shap_values_subset_for_decision = shap_values_positive_class[:num_decision_plots, :]
    features_subset_for_decision = X_test.iloc[:num_decision_plots]

    print(f"\nGenerazione del Decision Plot SHAP per le prime {num_decision_plots} istanze...")
    try:
        plt.figure(figsize=(10, 6))
        shap.decision_plot(expected_value_positive_class,
                           shap_values_subset_for_decision,
                           features_subset_for_decision,
                           feature_names=X_test_processed.columns.tolist(),
                           show=False,
                          )
        plt.title(f"SHAP Decision Plot per le prime {num_decision_plots} istanze (LightGBM)")
        plt.show()
        print(f"Decision Plot per le prime {num_decision_plots} istanze generato e mostrato.")
    except Exception as e:
        print(f"Errore durante la generazione del decision plot: {e}")
else:
    print(f"\nNon ci sono abbastanza istanze nel test set ({X_test_processed.shape[0]}) per generare il decision plot per {num_decision_plots} istanze.")

print("\n--- Analisi SHAP completata ---")